In [7]:
import torch
from torch.utils.data import DataLoader
from dataset import VitalDataset

trdt  = VitalDataset('../data/pd_gy','../data/pd_gy/train.json')
valdt = VitalDataset('../data/pd_gy','../data/pd_gy/val.json') 
tedt  = VitalDataset('../data/pd_gy','../data/pd_gy/test.json') 

trdl  = torch.utils.data.DataLoader(trdt,4)
valdl  = torch.utils.data.DataLoader(valdt,4)
tedl  = torch.utils.data.DataLoader(valdt,4)

trit = iter(trdl)
x,y  = next(trit)

In [25]:
import torchvision

from torch import nn
two23 = nn.Conv2d(2,3,3)

pre_backbone = torchvision.models.resnet34(pretrained=False)
post_backbone = torchvision.models.resnet34(pretrained=False)

In [56]:
pre_backbone.fc = nn.Sequential(nn.Linear(512,256),nn.Linear(256,128))
post_backbone.fc = nn.Sequential(nn.Linear(512,256),nn.Linear(256,128))

In [54]:
fusion = nn.Sequential(nn.Linear(256,128),nn.BatchNorm1d(128),nn.ReLU(),
                       nn.Linear(128,64), nn.BatchNorm1d(64), nn.ReLU(),
                       nn.Linear(64,2)) #

In [18]:
x0_0, x0_1 = x[:,:2],x[:,2:]

In [21]:
x0_0 = two23(x0_0)

x0_1 = two23(x0_1)

In [57]:
x0_0 = pre_backbone(x0_0)
x0_1 = post_backbone(x0_1)

In [63]:
x0 = torch.cat([x0_0,x0_1],axis=1)

In [ ]:
fusion(x0)

torch.Size([4, 2])

In [69]:
import torchvision
from torch import nn
import torch

class PrePostNet(nn.Module):

    def __init__(self):
        super(PrePostNet, self).__init__()
        
        self.two2three_pre  = nn.Conv2d(2,3,3)
        self.two2three_post = nn.Conv2d(2,3,3)
        
        self.pre_backbone = torchvision.models.resnet34(pretrained=True)
        self.post_backbone = torchvision.models.resnet34(pretrained=True)
        self.pre_backbone.fc = nn.Sequential(nn.Linear(512,256),nn.Linear(256,128))
        self.post_backbone.fc = nn.Sequential(nn.Linear(512,256),nn.Linear(256,128))
        
        self.fusion = nn.Sequential(nn.Linear(256,128),nn.BatchNorm1d(128),nn.ReLU(),
                       nn.Linear(128,64), nn.BatchNorm1d(64), nn.ReLU(),
                       nn.Linear(64,2)) 
        
    
    def forward(self, x):
        x_pre, x_post = x[:,:2], x[:,2:]
        
        x_pre  = self.two2three_pre(x_pre)
        x_post = self.two2three_post(x_post)
        
        x_pre  = self.pre_backbone(x_pre)
        x_post = self.post_backbone(x_post)
        
        x_fusion = torch.cat([x_pre,x_post],axis=1)
        x_fusion = self.fusion(x_fusion)
        
        return x_fusion

In [70]:
model = PrePostNet()
pre = model(x)

# test

In [1]:
import torch
from torch.utils.data import DataLoader
from dataset import VitalDataset

trdt  = VitalDataset('../data/pd_gy','../data/pd_gy/train.json')
valdt = VitalDataset('../data/pd_gy','../data/pd_gy/val.json') 
tedt  = VitalDataset('../data/pd_gy','../data/pd_gy/test.json') 

trdl  = torch.utils.data.DataLoader(trdt,4)
valdl  = torch.utils.data.DataLoader(valdt,4)
tedl  = torch.utils.data.DataLoader(valdt,4)

trit = iter(trdl)
x,y  = next(trit)

In [2]:
from models import PrePostNet

In [4]:
model = PrePostNet()

In [ ]:
model(x)

tensor([[-0.6482, -0.4512],
        [-0.2352,  0.1045],
        [ 0.6511, -0.2366],
        [-0.3394,  0.3636]], grad_fn=<AddmmBackward0>)